In [ ]:
!apt-get update
!apt-get install -y cuda


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,701 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,844 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubunt

In [12]:
from google.colab import files
uploaded = files.upload()

Saving graph_1000V_5000E.txt to graph_1000V_5000E.txt
Saving graph_2000V_10000E.txt to graph_2000V_10000E.txt
Saving graph_3000V_15000E.txt to graph_3000V_15000E.txt
Saving graph_4000V_20000E.txt to graph_4000V_20000E.txt
Saving graph_5000V_25000E.txt to graph_5000V_25000E.txt
Saving graph_6000V_30000E.txt to graph_6000V_30000E.txt
Saving graph_7000V_35000E.txt to graph_7000V_35000E.txt
Saving graph_8000V_40000E.txt to graph_8000V_40000E.txt


In [45]:
%%writefile bellman_ford_cuda.cu

// Necessary headers
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <sstream>
#include <algorithm>
#include <climits>
#include <cuda_runtime.h>

using namespace std;

#define INF (INT_MAX / 2)

// CUDA kernel to perform edge relaxation
__global__ void bellmanFordKernel(int* d_V, int* d_E, int* d_W, int* d_dist, int V, int E) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < E) {
        int u = d_V[idx];
        int v = d_E[idx];
        int w = d_W[idx];

        if (d_dist[u] != INF && d_dist[u] + w < d_dist[v]) {
            atomicMin(&d_dist[v], d_dist[u] + w);
        }
    }
}

// Function to write distances to a file
void writeToFile(const vector<int>& distances, const char *filename) {
    ofstream outfile(filename);
    for (int distance : distances) {
        if (distance >= INF / 2)  // Handle unreachable nodes
            outfile << "INF" << endl;
        else
            outfile << distance << endl;
    }
    outfile.close();
}

void readGraphFromFile(const char *filename, vector<int>& V, vector<int>& E, vector<int>& W, int& numVertices, int& numEdges) {
    ifstream infile(filename);
    if (!infile.is_open()) {
        cerr << "Error: Could not open file " << filename << endl;
        exit(1);
    }

    string line;

    // Read the first line for V and E
    if (!getline(infile, line)) {
        cerr << "Error: File is empty or first line is invalid." << endl;
        exit(1);
    }

    stringstream ss(line);
    ss >> numVertices >> numEdges;

    V.reserve(numEdges);
    E.reserve(numEdges);
    W.reserve(numEdges);

    int u, v, w;
    int edgeCount = 0;
    while (getline(infile, line)) {
        stringstream ss(line);
        if (!(ss >> u >> v >> w)) {
            cerr << "Warning: Invalid edge line ignored: " << line << endl;
            continue;
        }
        V.push_back(u);
        E.push_back(v);
        W.push_back(w);
        edgeCount++;
    }

    if (edgeCount != numEdges) {
        cerr << "Warning: Expected " << numEdges << " edges, but got " << edgeCount << "." << endl;
    }

    infile.close();
}

int main() {
    vector<int> h_V, h_E, h_W;
    int numVertices, numEdges;

    // Read the graph
    readGraphFromFile("graph_5000V_25000E.txt", h_V, h_E, h_W, numVertices, numEdges);

    cout << "Sample of first 5 edges(already 0-based)" << endl;
    for (int i = 0; i < min(5, (int)h_V.size()); ++i) {
        cout << h_V[i] << " -> " << h_E[i] << " [weight=" << h_W[i] << "]" << endl;
    }


    int V = numVertices;
    int E = numEdges;

    cout << "Graph loaded with " << V << " vertices and " << E << " edges." << endl;

    // === Detect source node with outgoing edges ===
    vector<vector<pair<int, int>>> adj(V);
    for (int i = 0; i < E; ++i) {
        int u = h_V[i];
        int v = h_E[i];
        int w = h_W[i];

        if (u < 0 || u >= V || v < 0 || v >= V) {
            cerr << "Invalid edge detected at index " << i << ": (" << u << " -> " << v << ")" << endl;
            continue;
        }

        adj[u].push_back({v, w});
    }


    int source = -1;
    for (int i = 0; i < V; ++i) {
        if (!adj[i].empty()) {
            cout << "Node " << (i + 1) << " has " << adj[i].size() << " outgoing edges." << endl;
            source = i;
            break;
        }
    }


    if (source == -1) {
        cerr << "No valid source node found!" << endl;
        return 1;
    }

    cout << "Detected source node: " << (source + 1) << " (0-based index = " << source << ")" << endl;

    // === Initialize distances ===
    vector<int> h_dist(V, INF);
    h_dist[source] = 0;

    // Device memory
    int *d_V, *d_E, *d_W, *d_dist;
    cudaMalloc(&d_V, E * sizeof(int));
    cudaMalloc(&d_E, E * sizeof(int));
    cudaMalloc(&d_W, E * sizeof(int));
    cudaMalloc(&d_dist, V * sizeof(int));

    cudaMemcpy(d_V, h_V.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_E, h_E.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_W, h_W.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, h_dist.data(), V * sizeof(int), cudaMemcpyHostToDevice);

    // Measure time for kernel execution
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    int threadsPerBlock = 256;
    int blocksPerGrid = (E + threadsPerBlock - 1) / threadsPerBlock;

    for (int i = 0; i < V - 1; ++i) {
        bellmanFordKernel<<<blocksPerGrid, threadsPerBlock>>>(d_V, d_E, d_W, d_dist, V, E);
        cudaDeviceSynchronize();
    }

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "CUDA Bellman-Ford execution time: " << milliseconds << " ms" << endl;

    cudaMemcpy(h_dist.data(), d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);

    writeToFile(h_dist, "shortest_path_distances.txt");

    cudaFree(d_V);
    cudaFree(d_E);
    cudaFree(d_W);
    cudaFree(d_dist);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting bellman_ford_cuda.cu


In [46]:
!nvcc -arch=sm_75 bellman_ford_cuda.cu -o bellman_ford_cuda

In [47]:
!./bellman_ford_cuda

Sample of first 5 edges(already 0-based)
4533 -> 1845 [weight=-4]
2740 -> 3122 [weight=-6]
422 -> 3491 [weight=12]
2852 -> 4485 [weight=17]
4576 -> 1566 [weight=19]
Graph loaded with 5000 vertices and 25000 edges.
Node 1 has 4 outgoing edges.
Detected source node: 1 (0-based index = 0)
CUDA Bellman-Ford execution time: 67.6087 ms


In [20]:
!ls

bellman_ford_cuda	graph_4000V_20000E.txt	real_life_weighted.txt
bellman_ford_cuda.cu	graph_5000V_25000E.txt	sample_data
graph_1000V_5000E.txt	graph_6000V_30000E.txt	shortest_path_distances.txt
graph_2000V_10000E.txt	graph_7000V_35000E.txt
graph_3000V_15000E.txt	graph_8000V_40000E.txt
